In [1]:
from boilerplates import *

23/05/08 23:02:17 WARN Utils: Your hostname, zylab resolves to a loopback address: 127.0.1.1; using 10.16.88.247 instead (on interface eno1)
23/05/08 23:02:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 23:02:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+-------------------+-------------------+--------+----------------+------------+------------+
|index|            in_time|           out_time|berthage|         section|admin_region|parking_time|
+-----+-------------------+-------------------+--------+----------------+------------+------------+
|    0|2018-09-01 10:10:00|2018-09-01 12:00:00|  201091|荔园路(蛇口西段)|      南山区|        6600|
|    1|2018-09-01 13:43:35|2018-09-01 14:29:35|  201091|荔园路(蛇口西段)|      南山区|        2760|
|    2|2018-09-01 15:10:54|2018-09-01 16:08:54|  201091|荔园路(蛇口西段)|      南山区|        3480|
|    3|2018-09-01 16:34:03|2018-09-01 17:56:03|  201091|荔园路(蛇口西段)|      南山区|        4920|
|    4|2018-09-01 18:40:20|2018-09-01 20:00:20|  201091|荔园路(蛇口西段)|      南山区|        4800|
+-----+-------------------+-------------------+--------+----------------+------------+------------+



/home/yecm/anaconda3/envs/spark/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark.version
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'test'),
 ('spark.sql.warehouse.dir',
  'file:/home/yecm/yecanming/repo/P_CS_Labs/Distrubuted/Assignment3-Spark/src/spark-warehouse'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '10.16.88.247'),
 ('spark.app.submitTime', '1683558138258'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar

In [3]:
# 问题1: 对于每个section，统计不同的停车场的数量（不是统计停车记录的数量，需要Distinct）
res1 = df.groupby(col_section).agg(count_distinct(col_berthage).alias("count"))
write_df(res1, "r1.csv")
# pandas得到单个文件。如果是write函数，可能得到多个文件。

written to /home/yecm/yecanming/repo/P_CS_Labs/Distrubuted/Assignment3-Spark/output/r1.csv


In [4]:
# 问题2
# 找到所有unique的berthage，同时找到每个berthage对应的那一个section
# 一个bertahge有很多条记录，每个记录都有section，我们选择哪个呢?
# 我们可以继续分组，看看是不是一样的。
res2 = df.select(col_berthage, col_section)\
    .groupby(col_berthage, col_section)\
    .agg(count_distinct(col_section).alias("count"))\
    # .agg(mode('section').alias('section'))
assert all([i['count']==1 for i in res2.toLocalIterator()])

In [5]:
# 符合条件，那么我们可以直接取第一个section
res2 = df.select(col_berthage, col_section)\
    .groupby(col_berthage)\
    .agg(first(col_section).alias('section'))
# res.show()
write_df(res2, "r2.csv")

written to /home/yecm/yecanming/repo/P_CS_Labs/Distrubuted/Assignment3-Spark/output/r2.csv


In [6]:
res2_5 = res2.groupby(col_section).agg(count(col_berthage).alias("count"))
dict(res2_5.toLocalIterator())==dict(res1.toLocalIterator()) # 说明r1和r2的正确性。

True

In [7]:
# 问题3
# 求每一个区域的平均停车时间，单位为取整后的秒
res3 = df.select(col_section, col_parking_time)\
    .groupby(col_section)\
    .agg(mean(col_parking_time).cast('long').alias('avg_parking_time'))
write_df(res3, "r3.csv")

written to /home/yecm/yecanming/repo/P_CS_Labs/Distrubuted/Assignment3-Spark/output/r3.csv


In [8]:
# 问题4
# 求每一个停车场的平均停车时间，单位为取整后的秒，降序排序
res4 = df.select(col_berthage, col_parking_time)\
    .groupby(col_berthage)\
    .agg(mean(col_parking_time).cast('long').alias('avg_parking_time'))\
    .sort(col('avg_parking_time').desc())
# res4.show()
write_df(res4, "r4.csv")


written to /home/yecm/yecanming/repo/P_CS_Labs/Distrubuted/Assignment3-Spark/output/r4.csv
